In [1]:
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = torch.device("cuda")

In [2]:
import sys
sys.path.append('./huggingface_models/')
sys.path.append('./utils/')
from sample_utils import *
from inference_utils import *
from codenet_process_utils import *
from self_training_utils import *

In [3]:
%load_ext autoreload
%autoreload 2

### Sampling:
1. get preds: get_preds_lang_dict_codenet
2. merge hypo files (since sampling takes time, we sample some languages in parallel. Thus we need to merge them later)

We get preds_lang_dict in this step. \
preds_lang_dict[(lang1, lang2)] = preds


### Cached Files
1. Preds from trained models\
    plbart_codenet_preds_lang_dict.pkl\
    codet5_codenet_preds_lang_dict.pkl

### Load No-tok Preprocessed Data

In [6]:
is_plbart = True
merged_filtered_dict = get_prepro_filtered_dict(None, is_plbart)
programs_dict = get_codenet_programs_dict(merged_filtered_dict)

### Pre-sampling

#### Get preds of unpaired programs

In [ ]:
script_path = "get_codenet_preds.py"

In [7]:
data_name = "codenet"
is_eval = False
sample_size = 5
temperature = 0.5
tag = "all"
model_type = "plbart"
exp_suffix = "_translation_exec_function/" 

In [ ]:
# get lang_pairs and preds
lang_pairs = []
for lang1 in new_langs:
    for lang2 in new_langs:
        if lang2 != lang1:
            lang_pairs.append((lang1, lang2))
# small_programs_dict = {x:["haha"] for x in new_langs}
preds_lang_dict = get_preds_lang_dict_codenet(lang_pairs, model_type, device, programs_dict, 
                                      sample_size, temperature, data_name, tag, exp_suffix)

#### Merge preds

In [ ]:
preds_lang_dict_all = {}
for lang in new_langs:
    dic_path = cached_path + "plbart_all_" + lang + "_codenet_preds_lang_dict.pkl"
    if not os.path.exists(dic_path):
        continue
    with open(dic_path, 'rb') as infile:
        preds_lang_dict_batch = pickle.load(infile)
        for k, v in preds_lang_dict_batch.items():
            print(lang, k)
            preds_lang_dict_all[k] = v
print(preds_lang_dict_all.keys())

In [5]:
plbart_sample_path = cached_path + 'plbart_codenet_preds_lang_dict.pkl'
with open(plbart_sample_path, 'rb') as infile:
     preds_lang_dict_plbart = pickle.load(infile)

In [7]:
codet5_sample_path = cached_path + 'codet5_codenet_preds_lang_dict.pkl'
with open(codet5_sample_path, 'rb') as infile:
     preds_lang_dict_codet5 = pickle.load(infile)